In [7]:
cd drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [8]:
!ls

BWeight.md5	       test_df.csv	train_df.csv.zip
radam_submission.csv   test_df.csv.zip	week7_keras_assignment_이지용.ipynb
sample_submission.csv  train_df.csv	week7_pytorch_assignment_이지용.ipynb


In [0]:
import pandas as pd
sample_submission = pd.read_csv("sample_submission.csv")
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")

In [0]:
X = train.iloc[:,1:].values / 255
y = train.iloc[:,0].values
X_test = test.iloc[:,1:].values / 255

In [11]:
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
y

array([7, 8, 7, ..., 9, 4, 3])

In [0]:
from torch.utils.data import Dataset, DataLoader

class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

class TestDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        return X

데이터를 batch 단위로 입력시키기 위한 batch iterator 생성  
모델의 파라미터를 업데이트 시키기 방법으로 gradient descent 방법 등을 사용합니다. 이 때 업데이트의 주기를  
1) 전체 데이터를 다 보고나서    
2) 데이터 하나하나를 볼 때마다  
3) 그 중간 단위로 batch size 만큼의 데이터를 보고나서 로 구분지을 수 있습니다.  
가장 결과가 잘 나오고, 효율적인 방법인 
3)을 주로 사용하므로 batch iterator을 만들 필요가 있겠죠.  
출처 : https://medium.com/@inmoonlight/pytorch%EB%A1%9C-%EB%94%A5%EB%9F%AC%EB%8B%9D%ED%95%98%EA%B8%B0-cnn-62a9326111ae

In [0]:
traindataloader = DataLoader(TrainDataset(X, y), batch_size=128, shuffle=True, num_workers=4)
testdataloader = DataLoader(TestDataset(X_test, y=None), batch_size=4, shuffle=False, num_workers=4)

class torch.nn.Module은 \_\_init\_\_ 과 forward를 정의해주어야 한다.  
Applies a linear transformation to the incoming data: y = xA^T + by=xA 
T
 +b

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)                  # 입력 픽셀 수가 784개
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)                   # 클래스는 10개
        self.elu = nn.ELU()
        self.dropout1 = torch.nn.Dropout(p=0.2)
        self.dropout2 = torch.nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(F.relu(x))
        x = self.dropout1(x)
        x = self.fc3(self.elu(x))
        x = self.dropout2(x)
        x = self.fc4(x)
        return x

    # weight initialization
    def init_weights(m):
      if type(m) == nn.Linear:
        torch.nn.init.kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='relu')  #he initialization

net = Net()

In [16]:
net

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
)

손실함수와 최적화 기법 설정

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

#optimizer = optim.Adagrad(net.parameters(), lr=2e-3, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10)
#optimizer = optim.Adam(net.parameters(), Ir=2e-3, weight_decay=1e-3)
optimizer = optim.SGD(net.parameters(), lr=2e-3, momentum=0.9)

디바이스를 GPU로 설정한다

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [19]:
net.to(device)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
)

In [29]:
num_epochs = 100
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(traindataloader):
        inputs, labels = data
        inputs = torch.tensor(inputs, device=device).float()
        labels = torch.tensor(labels, device=device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # print(type(inputs), type(labels), type(outputs))
        # forward + backward + optimize
        outputs = net(inputs)
        # print(type(inputs), type(labels), type(outputs))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    print("Epoch : {} loss: {}".format(epoch, running_loss))
print('Finished Training')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Epoch : 0 loss: 0.32470881193876266
Epoch : 1 loss: 0.31328876316547394
Epoch : 2 loss: 0.29739875346422195
Epoch : 3 loss: 0.2869708128273487
Epoch : 4 loss: 0.28977905213832855
Epoch : 5 loss: 0.2763303965330124
Epoch : 6 loss: 0.2747499383985996
Epoch : 7 loss: 0.3003668934106827
Epoch : 8 loss: 0.27326393127441406
Epoch : 9 loss: 0.2536405436694622
Epoch : 10 loss: 0.2886468879878521
Epoch : 11 loss: 0.286019753664732
Epoch : 12 loss: 0.2657979018986225
Epoch : 13 loss: 0.272006556391716
Epoch : 14 loss: 0.26823708787560463
Epoch : 15 loss: 0.28066450357437134
Epoch : 16 loss: 0.270059235394001
Epoch : 17 loss: 0.24658559262752533
Epoch : 18 loss: 0.2478296086192131
Epoch : 19 loss: 0.257940661162138
Epoch : 20 loss: 0.2719881907105446
Epoch : 21 loss: 0.2408161349594593
Epoch : 22 loss: 0.24821965023875237
Epoch : 23 loss: 0.23317361623048782
Epoch : 24 loss: 0.2507300339639187
Epoch : 25 loss: 0.24726209044456482
Epoch : 26 loss: 0.24584103003144264
Epoch : 27 loss: 0.23761586099

In [30]:
correct = 0
total = 0
net.eval()
preds = []
with torch.no_grad():
    for data in testdataloader:
        inputs = data
        inputs = torch.tensor(inputs, device=device).float()
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted:
          preds.append(pred.cpu().numpy())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [31]:
preds[:10]

[array(8),
 array(0),
 array(5),
 array(3),
 array(8),
 array(1),
 array(9),
 array(6),
 array(6),
 array(0)]

In [32]:
import numpy as np
preds = np.array(preds)
print(preds.shape)

(18000,)


In [33]:
sample_submission["Category"] = pd.Series(preds)
sample_submission.head()

,Id,Category
0,57808,8
1,4960,0
2,35755,5
3,15543,3
4,48968,8


In [0]:
sample_submission.to_csv("torch_submission.csv", index=False)